In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

In [ ]:
chunk_size = 10000
chunks = pd.read_csv("aws_review_sofware_dataset.csv", sep=',', chunksize=chunk_size)

try:
    #chunks = pd.read_csv(file_path, sep=',', chunksize=chunk_size)
    df = next(chunks)  # Get the first chunk
    print("Columns in the dataset:", df.columns)
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")


Columns in the dataset: Index(['review', 'sentiment'], dtype='object')


In [3]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [4]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")


In [5]:
from nltk.tokenize import sent_tokenize

In [6]:

import nltk
nltk.download('punkt')
#nltk.download('data')

[nltk_data] Downloading package punkt to C:\Users\D MADHAN
[nltk_data]     MOHAN/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
for i in range(df.shape[0]):
    l1= sent_tokenize(str(df.loc[i,"reviewText"]))
    df.at[i,"sentences"]=l1

In [78]:
!pip install pywsd

In [47]:

import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\D MADHAN MOHAN/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [48]:
from pywsd.utils import lemmatize_sentence

In [49]:
import nltk
#nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\D MADHAN
[nltk_data]     MOHAN/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [50]:
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Function to lemmatize sentences
def lemmatize_with_nltk(sentence):
    tokens = word_tokenize(sentence)
    return [lemmatizer.lemmatize(word) for word in tokens]

# Apply the custom lemmatizer
for k in range(df.shape[0]):
    df.at[k, "words"] = []
    for sentence in df.loc[k, "sentences"]:
        lemmatized_words = lemmatize_with_nltk(sentence)
        df.at[k, "words"].extend(lemmatized_words)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\D MADHAN MOHAN/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\Users\D MADHAN
[nltk_data]     MOHAN/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\D MADHAN
[nltk_data]     MOHAN/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [52]:
df["words_sentences"] = "default"

In [53]:
import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=functools.reduce(lambda a,b:( str(a)+str(" ")+str(b)),df.loc[k,"words"])

In [54]:
from sklearn.feature_extraction.text import  CountVectorizer

In [55]:
df1=df

no_features = 1000
tf_vectorizer = CountVectorizer( max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df1.words_sentences)

In [56]:
df_x=pd.DataFrame(tf.toarray(), columns=tf_vectorizer.get_feature_names_out())

In [57]:
df_y=df["verified"]

In [58]:

from sklearn.preprocessing import LabelEncoder

In [59]:

le=LabelEncoder()

In [60]:
df_y_1=pd.DataFrame(df_y)

In [61]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [62]:
df_y_enc.columns

Index(['verified'], dtype='object')

In [63]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=500, random_state=42)
rf.fit(df_x,df_y_enc)

# Accuracy
accuracy_rf = rf.score(df_x,df_y_enc)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")


c:\Users\D MADHAN MOHAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 99.93%


In [65]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(df_x,df_y_enc)

# Compute accuracy
accuracy_nb = nb.score(df_x,df_y_enc)
print(f"Naive Bayes Accuracy: {accuracy_nb * 100:.2f}%")


c:\Users\D MADHAN MOHAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Naive Bayes Accuracy: 70.74%


In [66]:
from sklearn.ensemble import GradientBoostingClassifier

In [67]:
GBC=GradientBoostingClassifier(n_estimators=1000)

In [68]:
gb_c=GBC.fit(df_x,df_y_enc)

c:\Users\D MADHAN MOHAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [69]:
gbc_score=GBC.score(df_x,df_y_enc)
print(f"gbc_score: {gbc_score* 100:.2f}%")

gbc_score: 89.09%
